# 라이브러리 Import

In [11]:
from torch.utils.data import Dataset
from PIL import Image
import cv2
import os
import math
import glob
import requests
from osgeo import gdal
import rasterio
from PIL import Image
import sys
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from segmentation_models_pytorch.utils.metrics import IoU
import wandb
import random
import torch.nn as nn
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from torchvision import transforms, utils
import shutil
from sklearn.model_selection import train_test_split
import warnings

# Config

In [12]:
# 데이터셋 경로
predict_img_path = '../dataset/test_img'
predict_pth_path = './model_save/best_model.pth'

# 생성 폴더 경로
predict_mask_path = './predict_mask'
pkl_save_path = './pkl_save'

os.makedirs(predict_mask_path, exist_ok=True)
os.makedirs(pkl_save_path, exist_ok=True)

In [13]:
# 예측 params
BATCH_SIZE = 16
IMG_SIZE = 256
CHANNELS = 3
NUM_WORKERS = 0

In [14]:
# 시드 설정
seed = 42

random.seed(seed)
np.random.seed(seed) 
os.environ["PYTHONHASHSEED"] = str(seed) 
torch.manual_seed(seed)  
torch.cuda.manual_seed(seed)  
torch.backends.cudnn.benchmark = False  

# 데이터셋 준비

In [15]:
def get_bounds(width, height, transform):

    left = int(float(transform[2]))
    right = int(float(transform[2])) + int(float(width))*int(float(transform[0]))
    bottom = int(float(transform[5])) + int(float(height))*int(float(transform[4]))
    top = int(float(transform[5]))

    bounds = (left, bottom, right, top)

    return bounds

def get_extent(dataset):

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()

    minx = transform[0]
    maxx = transform[0] + (cols * transform[1]) + (rows * transform[2])
    miny = transform[3] + (cols * transform[4]) + (rows * transform[5])
    maxy = transform[3]

    return {"minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy),
            "cols": str(cols), "rows": str(rows)}

def getReflectance (band, add_band, mult_band, sun_elevation):
    p = ((band * mult_band) + add_band)
    corrected = p / math.sin (math.radians (sun_elevation))

    return p, corrected

def get_saturation(BQA):
    vals = [2724,2756,2804,2980,3012,3748,3780,6820,6852,6900,7076,7108,7844,7876,
            2728,2760,2808,2984,3016,3752,3784,6824,6856,6904,7080,7112,7848,7880,
            2732,2764,2812,2988,3020,3756,3788,6828,6860,6908,7084,7116,7852,7884]
    
    sat = np.zeros((BQA.shape), dtype=bool)

    for val in vals:
        sat = sat | (BQA==val)
        
    return sat.astype(int)

def Seq1 (bands, r75, diff75):
    return (np.logical_and (bands [7] > 0.5, np.logical_and (r75 > 2.5, diff75 > 0.3)))

def Seq2 (bands):
    return (np.logical_and (bands [6] > 0.8, np.logical_and (bands [1] < 0.2, np.logical_or (bands [5] > 0.4, bands [7] < 0.1))))

def Seq3 (r75, diff75):
    return (np.logical_and (r75 > 1.8, diff75 > 0.17))

def Seq4and5 (bands, r75, unamb_fires, potential_fires, water):

    ignored_pixels = np.logical_or (bands [7] <= 0, np.logical_or (unamb_fires, water))
    kept_pixels = np.logical_not (ignored_pixels)

    r75_ignored = r75.copy ()
    r75_ignored [ignored_pixels] = np.nan

    band7_ignored = bands [7].copy ()
    band7_ignored [ignored_pixels] = np.nan

    candidates = np.nonzero (potential_fires)
    for i in range (len (candidates [0])):
        y = candidates [0][i]
        x = candidates [1][i]

        t = max (0,y-30)
        b = min (potential_fires.shape [0], y+31)
        l = max (0, x-30)
        r = min (potential_fires.shape [1], x+31)

        eq4_result = r75 [y,x] > np.nanmean (r75_ignored [t:b,l:r]) + np.maximum (3 * (np.nanstd (r75_ignored [t:b,l:r])), 0.8)
        eq5_result = bands [7][y,x] > np.nanmean (band7_ignored [t:b,l:r]) + np.maximum (3 * (np.nanstd (band7_ignored [t:b,l:r])), 0.08)
        if not (eq4_result) or not (eq5_result):
            potential_fires [y,x] = False

    return potential_fires

def Seq6 (bands):
    p6 = np.where (bands[6] == 0, np.finfo (float).eps, bands[6])
    return (bands [7] / p6 > 1.6)

def Seq7_8_9 (bands):
    result7 = np.logical_and (bands [4] > bands [5], np.logical_and (bands [5] > bands [6], np.logical_and (bands [6] > bands [7], bands [1] - bands [7] < 0.2)))
    return (np.logical_and (result7, np.logical_or (bands [3] > bands [2], np.logical_and (bands [1] > bands [2], np.logical_and (bands [2] > bands [3], bands [3] > bands [4])))))


def Geq12 (bands):
    return (bands [4] <= 0.53 * bands [7] - 0.214)

def Geq13 (bands, eq12_mask):
    neighborhood = cv2.dilate (eq12_mask.astype (np.uint8), cv2.getStructuringElement (cv2.MORPH_RECT, (3,3))).astype (eq12_mask.dtype)

    return (np.logical_and (neighborhood, bands [4] <= 0.35 * bands [6] - 0.044))

def Geq14 (bands):
    return (bands [4] <= 0.53 * bands [7] - 0.125)

def Geq15 (bands):
    return (bands [6] <= 1.08 * bands [7] - 0.048)

def Geq16 (bands):
    return (np.logical_and (np.logical_and (bands [2] > bands [3], bands [3] > bands [4]), bands [4] > bands [5]))

def pixelVal(p7,ef,ep,ew):
    e = np.logical_and (p7>0, np.logical_and (np.logical_not (ef), np.logical_and (np.logical_not (ep), np.logical_not (ew))))
    return e

def Geq8and9 (bands, valid, unamb_fires, potential_fires, water):

    ignored_pixels = np.logical_or (unamb_fires, np.logical_or (potential_fires, water))
    ignored_pixels = np.logical_or (ignored_pixels, np.logical_not (valid))
    kept_pixels = np.logical_not (ignored_pixels)

    r75 = bands [7] / bands [5]
    r75_ignored = r75.copy ()
    r75_ignored [ignored_pixels] = np.nan

    band7_ignored = bands [7].copy ()
    band7_ignored [ignored_pixels] = np.nan

    sizes = list(range(5,61+2,2))

    candidates = np.nonzero (potential_fires)

    for i in range (len (candidates [0])):
        y = candidates [0][i]
        x = candidates [1][i]
        tested = False
        for w in sizes:
            t = max (0,y-w//2)
            b = min (potential_fires.shape [0], y+w//2+1)
            l = max (0, x-w//2)
            r = min (potential_fires.shape [1], x+w//2+1)

            if np.count_nonzero (kept_pixels [t:b,l:r]) >= 0.25 * (b-t)*(r-l):
                tested = True
                eq8_result = r75 [y,x] > np.nanmean (r75_ignored [t:b,l:r]) + np.maximum (3 * (np.nanstd (r75_ignored [t:b,l:r])), 0.8)
                eq9_result = bands [7][y,x] > np.nanmean (band7_ignored [t:b,l:r]) + np.maximum (3 * (np.nanstd (band7_ignored [t:b,l:r])), 0.08)
                if not (eq8_result) or not (eq9_result):
                    potential_fires [y,x] = False
                break

        if not tested:
            potential_fires [y,x] = False

    return potential_fires

def Meq2 (bands):

    p5 = np.where (bands[5] == 0, np.finfo (float).eps, bands[5])
    p6 = np.where (bands[6] == 0, np.finfo (float).eps, bands[6])
    
    return (np.logical_and (bands[7] >= 0.15, np.logical_and (bands[7]/p6 >= 1.4, bands[7]/p5 >= 1.4)))

def Meq3 (bands, unamb, sat):

    neighborhood = cv2.dilate (unamb.astype (np.uint8), cv2.getStructuringElement (cv2.MORPH_RECT, (3,3))).astype (unamb.dtype)
    p5 = np.where (bands[5] > 0, np.finfo (float).eps, bands[5])
    
    return (np.logical_and (neighborhood, np.logical_or (np.logical_and (bands[6]/p5 >= 2.0, bands[6]>=0.5), sat)))

def getFireGOLI (bands):

    valid = bands [7] > 0
    valid = cv2.erode (valid.astype (np.uint8), cv2.getStructuringElement (cv2.MORPH_RECT, (3,3))).astype (np.uint8)

    unamb_fires = Geq12 (bands)
    unamb_fires = np.logical_and (valid, unamb_fires)
    if np.any (unamb_fires):
        unamb_fires = np.logical_or (unamb_fires, Geq13 (bands, unamb_fires))
        unamb_fires = np.logical_and (valid, unamb_fires)

    potential_fires = Geq14 (bands)
    potential_fires = np.logical_or (potential_fires, Geq15 (bands))
    potential_fires = np.logical_and (valid, potential_fires)

    water = Geq16 (bands)

    if np.any (potential_fires):
        potential_fires = Geq8and9 (bands, valid, unamb_fires, potential_fires, water)

    scaled_band = np.logical_and (np.logical_or (unamb_fires, potential_fires), np.logical_not (water))
    return (scaled_band.astype (int))

def getFireMurphy (bands, saturated):
    unamb_fires = Meq2 (bands)

    if np.any (unamb_fires):
        potential_fires = Meq3 (bands, unamb_fires, saturated)
        scaled_band = (unamb_fires | potential_fires)
    else:
        scaled_band = unamb_fires

    return (scaled_band.astype (int))

def getFireSchroeder (bands):
    r75 = bands [7] / bands [5]
    diff75 = bands [7] - bands [5]

    unamb_fires = Seq1 (bands, r75, diff75)
    unamb_fires = np.logical_or (unamb_fires, Seq2 (bands))

    potential_fires = Seq3 (r75, diff75)

    potential_fires = np.logical_and (potential_fires, Seq6 (bands))

    water = Seq7_8_9 (bands)

    if np.any (potential_fires):
        potential_fires = Seq4and5 (bands, r75, unamb_fires, potential_fires, water)

    scaled_band = np.logical_and (np.logical_or (unamb_fires, potential_fires), np.logical_not (water))
    return (scaled_band.astype (int))

def processImage (in_dir, image_name, Aref, Mref, SE, sat):
    bands = np.zeros((8, 256, 256))
    
    with rasterio.open (os.path.join (in_dir, image_name + '.tif')) as src:
        profile = src.profile.copy ()
        for i in range (8):
            if i == 0:
                pass
            else:
                bands[i] = src.read (i)
                
    reflectance = np.copy(bands)
    corrected = np.copy(bands)
    
    for i in range (1,8):
        reflectance[i], corrected[i] = getReflectance (bands[i], Aref[i-1], Mref[i-1], SE)
        
    scaled_schroeder = getFireSchroeder(reflectance)
    scaled_goli = getFireGOLI(corrected)
    scaled_murphy = getFireMurphy(corrected, sat)

    scaled_ir_band = [scaled_schroeder, scaled_goli, scaled_murphy]
    
    return np.array(scaled_ir_band)

def voted_image(scaled_ir_band):
    ir_voted_band = np.sum(scaled_ir_band, axis=0) >= 2
    ir_voted_band = ir_voted_band.astype(np.uint8)
    
    return ir_voted_band

def bands_combine(in_dir, image_name, scaling_band, g_band, b_band):
    tif_file = os.path.join (in_dir, image_name + '.tif')
    tif_data = gdal.Open(tif_file)
    
    band_b = (tif_data.GetRasterBand(1).ReadAsArray() / 256).astype(np.uint8)
    band_g = (tif_data.GetRasterBand(2).ReadAsArray() / 256).astype(np.uint8)
    band_r = scaling_band * 255
    
    rgb_image = np.dstack((band_b, band_g, band_r)).astype(np.uint8)
    
    return rgb_image

def adjust_contrast(image, contrast=1.0):
    f = 131 * (contrast + 127) / (127 * (131 - contrast))
    alpha_c = f
    gamma_c = 127 * (1 - f)
    return cv2.addWeighted(image, alpha_c, image, 0, gamma_c)

In [16]:
class CustomDataGenerator(Dataset):
    def __init__(self, images_dir, transform=None, percent=1, ASSUMED_SE=55, ASSUMED_MREF=None, ASSUMED_AREF=None):
        self.images_dir = images_dir
        self.images = sorted([x for x in os.listdir(images_dir) if x.endswith('.tif')])[:int(len(os.listdir(images_dir)) * percent)]
        self.transform = transform
        self.ASSUMED_SE = ASSUMED_SE
        self.ASSUMED_MREF = [2e-05] * 8 if ASSUMED_MREF is None else ASSUMED_MREF
        self.ASSUMED_AREF = [-0.1] * 8 if ASSUMED_AREF is None else ASSUMED_AREF

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        tif_path = os.path.join(self.images_dir, self.images[idx])

        image_name = os.path.basename(tif_path.replace('.tif',''))
        with rasterio.open(tif_path) as src:
            profile = src.profile.copy()
            BQA = src.read(1)
        saturation = get_saturation(BQA)  
        scaling_band_3ch = processImage(self.images_dir, image_name, self.ASSUMED_AREF, self.ASSUMED_MREF, self.ASSUMED_SE, saturation)
        scaling_band = voted_image(scaling_band_3ch)
        img_array = bands_combine(self.images_dir, image_name, scaling_band, 2, 1)
        img_array = adjust_contrast(img_array, contrast=1.3)
        img = Image.fromarray(img_array).convert("RGB")
        
        if self.transform:
            img = self.transform(img)

        png_filename = self.images[idx].replace('.tif', '.png')

        return {'image': img, 'name': png_filename}

# Predict

In [17]:
class ConvBlock(nn.Module):
    def __init__(self, channel_in, channel_out, dilation=1):
        super().__init__()
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=3, padding='same', dilation=dilation)
        self.conv2 = nn.Conv2d(channel_out, channel_out, kernel_size=3, padding='same', dilation=dilation)
        self.bnorm1 = nn.BatchNorm2d(channel_out)
        self.bnorm2 = nn.BatchNorm2d(channel_out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv1 = self.activation(self.bnorm1(conv1))
        conv2 = self.conv2(conv1)
        conv2 = self.activation(self.bnorm2(conv2))
        return conv2

class Downsample(nn.Module):
    def __init__(self, channel_in):
        super().__init__()
        self.downsample = nn.Conv2d(channel_in, channel_in * 2, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        return self.downsample(x)

class Upsample(nn.Module):
    def __init__(self, channel_in, channel_out):
        super().__init__()
        self.conv_transpose = nn.ConvTranspose2d(channel_in, channel_out, kernel_size=2, stride=2)

    def forward(self, x):
        return self.conv_transpose(x)

class UNet(nn.Module):
    def __init__(self, clannels, classes):
        super(UNet, self).__init__()
        self.CHANNELS = clannels
        self.CLASSES = classes

        self.inp = ConvBlock(self.CHANNELS, 64)

        self.stage1 = ConvBlock(128, 128, dilation=1)
        self.stage2 = ConvBlock(256, 256, dilation=1)
        self.stage3 = ConvBlock(512, 512, dilation=2)
        self.stage4 = ConvBlock(1024, 1024, dilation=3)

        self.down1 = Downsample(64)
        self.down2 = Downsample(128)
        self.down3 = Downsample(256)
        self.down4 = Downsample(512)

        self.up1 = Upsample(1024, 512)
        self.up2 = Upsample(512, 256)
        self.up3 = Upsample(256, 128)
        self.up4 = Upsample(128, 64)

        self.stage4i = ConvBlock(1024, 512, dilation=3)
        self.stage3i = ConvBlock(512, 256, dilation=2)
        self.stage2i = ConvBlock(256, 128, dilation=1)
        self.stage1i = ConvBlock(128, 64, dilation=1)

        self.out = nn.Conv2d(64, self.CLASSES, kernel_size=1)

    def forward(self, x):
        a1 = self.inp(x)
        d1 = self.down1(a1)

        a2 = self.stage1(d1)
        d2 = self.down2(a2)

        a3 = self.stage2(d2)
        d3 = self.down3(a3)

        a4 = self.stage3(d3)
        d4 = self.down4(a4)

        a5 = self.stage4(d4)
        u1 = self.up1(a5)

        c1 = self.stage4i(torch.cat([a4, u1], dim=1))
        u2 = self.up2(c1)

        c2 = self.stage3i(torch.cat([a3, u2], dim=1))
        u3 = self.up3(c2)

        c3 = self.stage2i(torch.cat([a2, u3], dim=1))
        u4 = self.up4(c3)

        c4 = self.stage1i(torch.cat([a1, u4], dim=1))
        logits = self.out(c4)

        return logits

def unet(n_channels=3, n_classes=1):
    return UNet(n_channels, n_classes)

In [18]:
def save_image(tensor, filename, suffix):
    filename = os.path.join(predict_mask_path, f"{filename.split('.')[0]}.png")
    utils.save_image(tensor, filename)

def predict(model):
    testDataset = CustomDataGenerator(
        images_dir=predict_img_path,
        transform=transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
        ])
    )

    testLoader = DataLoader(testDataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=NUM_WORKERS)

    model.eval()
    with torch.no_grad():
        for batch in tqdm(testLoader):
            images = batch['image'].to(device)
            filenames = batch['name']
            outputs = model(images)
            
            # 예측 결과 저장
            for i, filename in enumerate(filenames):
                save_image(outputs[i].cpu(), filename, 'pred')

In [19]:
# wandb.init(project=wandb_project_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using:', device)

Using: cuda


In [20]:
model = unet(n_channels=CHANNELS).to(device)
# model = torch.nn.DataParallel(model).to(device)

if os.path.exists(predict_pth_path):
    checkpoint = torch.load(predict_pth_path)
    model.load_state_dict(checkpoint)
else:
    print("Not Exists Model")

warnings.filterwarnings('ignore')
predict(model)

100%|██████████| 250/250 [02:47<00:00,  1.50it/s]
